## Tuto

In [3]:
import gym

from stable_baselines3 import A2C

env = gym.make('CartPole-v1')

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

obs = env.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 27.4     |
|    ep_rew_mean        | 27.4     |
| time/                 |          |
|    fps                | 3282     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.667   |
|    explained_variance | -0.487   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -3.83    |
|    value_loss         | 47.3     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 22.1     |
|    ep_rew_mean        | 22.1     |
| time/                 |          |
|    fps                | 3230     |
|    iterations         | 200      |
|    time_elapsed 

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 29.7     |
|    ep_rew_mean        | 29.7     |
| time/                 |          |
|    fps                | 3252     |
|    iterations         | 1400     |
|    time_elapsed       | 2        |
|    total_timesteps    | 7000     |
| train/                |          |
|    entropy_loss       | -0.381   |
|    explained_variance | 9.06e-05 |
|    learning_rate      | 0.0007   |
|    n_updates          | 1399     |
|    policy_loss        | 1.81     |
|    value_loss         | 2.66     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 31.7     |
|    ep_rew_mean        | 31.7     |
| time/                 |          |
|    fps                | 3254     |
|    iterations         | 1500     |
|    time_elapsed       | 2        |
|    total_timesteps    | 7500     |
| train/                |          |
|

KeyboardInterrupt: 

In [2]:
from stable_baselines3 import A2C

model = A2C('MlpPolicy', 'CartPole-v1').learn(10000)

## Setup

In [10]:
import numpy as np

import gym
from gym import spaces

from stable_baselines.common.env_checker import check_env
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.policies import MlpPolicy

## Environement

In [11]:
grid_size = 5
n_actions = grid_size * grid_size

In [ ]:
class CustomEnv(gym.Env):
  """Custom Environment that follows gym interface"""
  metadata = {'render.modes': ['human']}

    def __init__(self, arg1, arg2, ...):
        super(CustomEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        # Example for using image as input:
        self.observation_space = spaces.Box(low=0, high=255,
                                        shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)

    def step(self, action):
        pass
        return observation, reward, done, info
    
    
    def reset(self):
        pass
        return observation  # reward, done, info can't be included
    
    
    def render(self, mode='human'):
        pass
        
        
    def close (self):
        pass
        

In [ ]:
class IArchitectEnv(py_environment.PyEnvironment):
    def __init__(self,verbose=False):
        self._state = np.zeros((nb_actions,),dtype=np.int32)
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=nb_actions-1, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(self._state.shape[0],), dtype=np.int32, minimum=0, name='observation')
        
        self._episode_ended = False
        self.verbose = verbose
        self.REWARD = 100
        self.PENALTY_ALREADY_FILLED = -100
        self.DEFAULT_REWARD = 1

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._state = np.zeros((nb_actions,),dtype=np.int32)
        self._episode_ended = False
        return ts.restart(self._state)
#         return ts.restart(np.array([self._state], dtype=np.int32))


    def _step(self, action):
        """
        Remplit la action_ième case
        Retour une penalité si case est déjà remplie
        """
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()

        # Make sure episodes don't go on forever.
        action_ = action.round().astype(int)
        assert action_ in list(range(self._state.shape[0]))
        
        if self.verbose:
            print(self._state,action,action_)

        reward = self.DEFAULT_REWARD + self._state.sum()
        
        if self._state[action_]==1:
            reward = self.PENALTY_ALREADY_FILLED
            self._episode_ended = True

        self._state[action_]=1

        if self._state.sum()==self._state.shape[0]:
            self._episode_ended = True
            reward = self.REWARD
        
        if not self._episode_ended:
             result = ts.transition(
                self._state, reward=reward, discount=1)
#                 np.array([self._state], dtype=np.int32), reward=penalty, discount=0.95)
        
        else:
            result = ts.termination(self._state, reward)
#             return ts.termination(np.array([self._state], dtype=np.int32), reward)
        if self.verbose:
            print(result.observation,reward)
        return result


In [25]:
class GoLeftEnv(gym.Env):
    """
    Custom Environment that follows gym interface.
    This is a simple env where the agent must learn to go always left. 
    """
    # Because of google colab, we cannot implement the GUI ('human' render mode)
    metadata = {'render.modes': ['console']}
    # Define constants for clearer code
    LEFT = 0
    RIGHT = 1

    def __init__(self, grid_size=10):
        super(GoLeftEnv, self).__init__()

        self.grid_size = grid_size
        self.agent_pos = grid_size - 1
        self.action_space = spaces.Discrete(n_actions)
        self.observation_space = spaces.Box(low=0, high=self.grid_size,
                                        shape=(5,5), dtype=np.int32)

    def reset(self):
        """
        Important: the observation must be a numpy array
        :return: (np.array) 
        """
        # Initialize the agent at the right of the grid
        self.agent_pos = self.grid_size - 1
        # here we convert to float32 to make it more general (in case we want to use continuous actions)
        return np.array([self.agent_pos]).astype(np.float32)

    def step(self, action):
        
        if action == self.LEFT:
            self.agent_pos -= 1
        elif action == self.RIGHT:
            self.agent_pos += 1
        else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))

        # Account for the boundaries of the grid
        self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size)

        # Are we at the left of the grid?
        done = bool(self.agent_pos == 0)

        # Null reward everywhere except when reaching the goal (left of the grid)
        reward = 1 if self.agent_pos == 0 else 0

        # Optionally we can pass additional info, we are not using that for now
        info = {}

        return np.array([self.agent_pos]).astype(np.float32), reward, done, info

    
    def render(self, mode='console'):
        if mode != 'console':
            raise NotImplementedError()
        # agent is represented as a cross, rest as a dot
        print("." * self.agent_pos, end="")
        print("x", end="")
        print("." * (self.grid_size - self.agent_pos))
        print(reward)

    def close(self):
        pass

## Test Random Environment

In [27]:
env = GoLeftEnv()
check_env(env, warn=True)

In [28]:
#environment = IArchitectEnv()
#env = gym.make(environment)

In [29]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

env.close()

.........x.
1
........x..
0
.......x...
0
........x..
0
.........x.
0
........x..
0
.......x...
0
......x....
0
.......x...
0
........x..
0
.......x...
0
........x..
0
.........x.
0
..........x
0
.........x.
0
..........x
0
.........x.
0
..........x
0
.........x.
0
..........x
0
..........x
0
..........x
0
.........x.
0
..........x
0
.........x.
0
........x..
0
.......x...
0
......x....
0
.......x...
0
......x....
0
.......x...
0
........x..
0
.......x...
0
........x..
0
.......x...
0
......x....
0
.......x...
0
......x....
0
.......x...
0
......x....
0
.......x...
0
......x....
0
.......x...
0
........x..
0
.......x...
0
......x....
0
.....x.....
0
....x......
0
.....x.....
0
....x......
0
...x.......
0
....x......
0
.....x.....
0
......x....
0
.......x...
0
........x..
0
.........x.
0
..........x
0
.........x.
0
..........x
0
..........x
0
..........x
0
..........x
0
..........x
0
.........x.
0
..........x
0
.........x.
0
........x..
0
.......x...
0
........x..
0
.......x...
0
......

....x......
0
.....x.....
0
......x....
0
.......x...
0
......x....
0
.......x...
0
......x....
0
.......x...
0
......x....
0
.....x.....
0
......x....
0
.....x.....
0
......x....
0
.......x...
0
......x....
0
.....x.....
0
......x....
0
.......x...
0
......x....
0
.....x.....
0
....x......
0
.....x.....
0
......x....
0
.....x.....
0
....x......
0
.....x.....
0
......x....
0
.....x.....
0
....x......
0
...x.......
0
....x......
0
.....x.....
0
......x....
0
.......x...
0
........x..
0
.......x...
0
........x..
0
.......x...
0
........x..
0
.........x.
0
........x..
0
.......x...
0
......x....
0
.......x...
0
........x..
0
.........x.
0
..........x
0
..........x
0
..........x
0
.........x.
0
........x..
0
.........x.
0
........x..
0
.........x.
0
..........x
0
..........x
0
.........x.
0
........x..
0
.........x.
0
........x..
0
.........x.
0
........x..
0
.......x...
0
......x....
0
.....x.....
0
......x....
0
.......x...
0
......x....
0
.......x...
0
......x....
0
.......x...
0
......

....x......
0
...x.......
0
..x........
0
.x.........
0
Episode:9 Score:1
.........x.
1
..........x
0
.........x.
0
..........x
0
..........x
0
..........x
0
.........x.
0
........x..
0
.......x...
0
......x....
0
.......x...
0
........x..
0
.......x...
0
......x....
0
.......x...
0
......x....
0
.......x...
0
........x..
0
.......x...
0
......x....
0
.......x...
0
........x..
0
.........x.
0
..........x
0
.........x.
0
........x..
0
.......x...
0
........x..
0
.........x.
0
........x..
0
.......x...
0
........x..
0
.......x...
0
........x..
0
.........x.
0
..........x
0
..........x
0
..........x
0
.........x.
0
........x..
0
.........x.
0
........x..
0
.......x...
0
........x..
0
.........x.
0
..........x
0
..........x
0
..........x
0
.........x.
0
........x..
0
.......x...
0
......x....
0
.....x.....
0
....x......
0
.....x.....
0
......x....
0
.....x.....
0
......x....
0
.......x...
0
........x..
0
.........x.
0
........x..
0
.......x...
0
........x..
0
.........x.
0
........x..
0
..

## Build and Train the Model

In [ ]:
env = gym.make(environment_name)
model = ACER('MlpPolicy', env, verbose = 1)

In [ ]:
model.learn(total_timesteps=100000)

## Save and Test the Model

In [ ]:
# Delete
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

In [ ]:
# Save the model
model.save("ACER_model")

In [2]:
# Delete the model
del model

In [ ]:
# Load the model
model = ACER.load("ACER_model", env=env)

In [ ]:
# Reset and test the model
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()